In [1]:
import pandas as pd
import sklearn as sk
import numpy as np
from sklearn.model_selection import train_test_split
vis_data = pd.read_csv("./train.csv", encoding = 'ISO-8859-1', low_memory = False)

In [2]:
df = pd.read_csv("train.csv", encoding = 'ISO-8859-1', low_memory = False)

In [3]:
df.head()

,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,clean_up_cost,judgment_amount,payment_amount,balance_due,payment_date,payment_status,collection_status,grafitti_status,compliance_detail,compliance
0,22056,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","INVESTMENT INC., MIDWEST MORTGAGE",2900.0,TYLER,NaN,3.0,S. WICKER,CHICAGO,...,0.0,305.0,0.0,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0
1,27586,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Michigan, Covenant House",4311.0,CENTRAL,NaN,2959.0,Martin Luther King,Detroit,...,0.0,855.0,780.0,75.0,2005-06-02 00:00:00,PAID IN FULL,NaN,NaN,compliant by late payment within 1 month,1.0
2,22062,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","SANDERS, DERRON",1449.0,LONGFELLOW,NaN,23658.0,P.O. BOX,DETROIT,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN
3,22084,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","MOROSI, MIKE",1441.0,LONGFELLOW,NaN,5.0,ST. CLAIR,DETROIT,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN
4,22093,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","NATHANIEL, NEAL",2449.0,CHURCHILL,NaN,7449.0,CHURCHILL,DETROIT,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN


In [4]:
df = df[['fine_amount','state_fee','late_fee','discount_amount','balance_due','compliance']]

In [5]:
df = df.dropna(axis = 0)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159880 entries, 0 to 250293
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   fine_amount      159880 non-null  float64
 1   state_fee        159880 non-null  float64
 2   late_fee         159880 non-null  float64
 3   discount_amount  159880 non-null  float64
 4   balance_due      159880 non-null  float64
 5   compliance       159880 non-null  float64
dtypes: float64(6)
memory usage: 8.5 MB


In [7]:
X = df[['fine_amount', 'state_fee', 'late_fee', 'discount_amount','balance_due']]
Y = df['compliance']

In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y, test_size = 0.3, shuffle = False)

In [9]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=23)
clf.fit(X_train, Y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=23, splitter='best')

In [10]:
from sklearn.metrics import f1_score
y_pred = clf.predict(X_train)
f1_train = f1_score(Y_train, y_pred)
print('[train] F1-score = {:.2f}'.format(f1_train))

[train] F1-score = 0.66


In [11]:
y_pred = clf.predict(X_test)
f1_val = f1_score(Y_test, y_pred)
print('[valid] F1-score = {:.2f}'.format(f1_val))

[valid] F1-score = 0.58


In [12]:
result = f1_train - f1_val

In [13]:
result

0.08055341409051786

Затем сделайте эту же выборку сбалансированной с помощью undersampling.

Для этого посчитайте количество примеров (n) класса-меньшинства, затем из класса-большинства возьмите n первых примеров.

То есть, в терминологии Python, возьмите срез (slice) от начала и до n, где n — количество примеров класса, которого в выборке представлено меньше.

Соедините две части выборки (с уменьшенным классом-большинством и с изначальным классом-меньшинством), сделайте точно такое же разбиение, как в задаче выше, и обучите такую же модель.

Посчитайте значение метрики  на новой тестовой выборке с помощью новой модели. Вычтите из значения метрики на тестовой части сбалансированной выборки значение метрики на тестовой части несбалансированной выборки и запишите результат в переменную result.

In [14]:
vis_data = pd.read_csv("train.csv", encoding = 'ISO-8859-1', low_memory = False)
vis_data_small = vis_data[['fine_amount', 'state_fee', 'late_fee', 
                           'discount_amount', 'balance_due', 'compliance']].dropna()
X = vis_data_small[['fine_amount', 'state_fee', 'late_fee', 'discount_amount', 'balance_due']]
y = vis_data_small['compliance']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)
clf = DecisionTreeClassifier(random_state=23)
clf.fit(X_train, y_train)
f1_imbalanced = f1_score(y_test, clf.predict(X_test))
zeros = y[y == 0]
ones = y[y == 1]
y_down = zeros[:len(ones)].append(ones)
X_down = X.loc[y_down.index]
X_train, X_test, y_train, y_test = train_test_split(X_down, y_down, test_size=0.3, shuffle=False)
clf = DecisionTreeClassifier(random_state=23)
clf.fit(X_train, y_train)
f1_balanced = f1_score(y_test, clf.predict(X_test))
result = f1_balanced - f1_imbalanced  

In [15]:
result

0.18272749967696011